In [1]:
import numpy as np
import matplotlib.pyplot as plt

# LSTM
#### Notation

$$x_j^{(i)[l]\langle t \rangle}$$
- $[l]$: layer l-th (L layers)
+ $(i)$: data point i-th (m data points)
+ $\langle t \rangle$: $\text{t}^\text{th}$ timestep (T timesteps)
+ $j$: feature j-th (n features)

$$[a, x]:\ \text{concatenate a with x}$$

# 1. Forward

## 1.1 LSTM Cell
<img src="./assets/LSTM_cell.png" width="550"/>

#### Forget gate $\mathbf{\Gamma}_{f}$

$$\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)\tag{1} $$


- Forget gate = a tensor containing values that are between 0 and 1 (sigmoid)
    + $\approx 0$ = forget/supress corresponding values in $\mathbf{c}^{\langle t-1 \rangle}$
    + $\approx 1$ = Remember corresponding values in $\mathbf{c}^{\langle t-1 \rangle}$
- Example: an LSTM to keep track of grammatical structures
    + if a subject change from singular ("puppy") to plural ("puppies")
    + the memory of the previous state becomes outdated -> forget that state

- Note:
    + $\mathbf{\Gamma}_f^{\langle t \rangle} * \mathbf{c}^{\langle t-1 \rangle}$ = applying a mask over the previous cell state.

#### Update gate $\mathbf{\Gamma}_{i}$

$$\mathbf{\Gamma}_i^{\langle t \rangle} = \sigma(\mathbf{W}_i[a^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)\tag{2} $$ 

- Update gate decide which parts of the candidate tensor $\tilde{\mathbf{c}}^{\langle t \rangle}$ to add to the cell state $c^{\langle t \rangle}$
- Update gate = a tensor containing values between 0 and 1 (sigmoid)
    + $\approx 0$ = Prevents corresponding values in $\tilde{\mathbf{c}}^{\langle t \rangle}$ from being passed onto the hidden state $\mathbf{c}^{\langle t \rangle}$
    + $\approx 1$ = Allow corresponding values in $\tilde{\mathbf{c}}^{\langle t \rangle}$ to be passed onto the hidden state $\mathbf{c}^{\langle t \rangle}$

- Note:
    + $\mathbf{\Gamma}_{i}^{\langle t \rangle} * \tilde{c}^{\langle t \rangle}$ is used in determining the cell state $\mathbf{c}^{\langle t \rangle}$.
    
#### Candidate value $\tilde{\mathbf{c}}^{\langle t \rangle}$

$$\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{a}^{\langle t - 1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right) \tag{3}$$

- The candidate value = a tensor containing information from the current time step that **may** be stored in the current cell state $\mathbf{c}^{\langle t \rangle}$.
    + Which parts of the candidate value get passed on depends on the update gate.
+ The candidate value = a tensor containing values that range from -1 to 1. (tanh)

#### Output gate $\mathbf{\Gamma}_{o}$
$$ \mathbf{\Gamma}_o^{\langle t \rangle}=  \sigma(\mathbf{W}_o[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})\tag{4}$$ 

- Output gate = a tensor contains values that range from 0 to 1 (sigmoid)

- Output gate determined by the previous hidden state $\mathbf{a}^{\langle t-1 \rangle}$ and the current input $\mathbf{x}^{\langle t \rangle}$

#### Cell (Memory) state $\mathbf{c}^{\langle t \rangle}$

$$ \mathbf{c}^{\langle t \rangle} = \mathbf{\Gamma}_f^{\langle t \rangle}* \mathbf{c}^{\langle t-1 \rangle} + \mathbf{\Gamma}_{i}^{\langle t \rangle} *\mathbf{\tilde{c}}^{\langle t \rangle} \tag{5} $$

- The cell state is the memory that gets passed onto future time steps.
- The previous cell state $\mathbf{c}^{\langle t-1 \rangle}$ is adjusted (weighted) by the forget gate $\mathbf{\Gamma}_{f}^{\langle t \rangle}$
- The candidate value $\tilde{\mathbf{c}}^{\langle t \rangle}$, adjusted (weighted) by the update gate $\mathbf{\Gamma}_{i}^{\langle t \rangle}$




#### Hidden state $\mathbf{a}^{\langle t \rangle}$

$$ \mathbf{a}^{\langle t \rangle} = \mathbf{\Gamma}_o^{\langle t \rangle} * \tanh(\mathbf{c}^{\langle t \rangle})\tag{6} $$

- The hidden state $\mathbf{a}^{\langle t \rangle}$ is determined by the cell state $\mathbf{c}^{\langle t \rangle}$ and the output gate $\mathbf{\Gamma}_{o}$
- The hidden state determines
    + $\mathbf{\Gamma}_{f}, \mathbf{\Gamma}_{u}, \mathbf{\Gamma}_{o}$ in the next time step
    + the prediction $y^{\langle t \rangle}$ in the current timestep
- Note
    + The cell state $\mathbf{c}^{\langle t \rangle}$ is passed through the "tanh" function to rescale values between -1 and +1.
    + The output gate acts like a "mask" to the values of $\tanh(\mathbf{c}^{\langle t \rangle})$
    
#### Prediction $\mathbf{\hat{y}}^{\langle t \rangle}$

$$\mathbf{\hat{y}}^{\langle t \rangle} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{a}^{\langle t \rangle} + \mathbf{b}_{y})$$

- Depend on the application to choose the activation. In this case classification -> softmax

In [2]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def softmax(Z):
    ez = np.exp(Z)
    return ez / np.sum(ez, axis=0)

In [3]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Arguments:
        xt (ndarray (n_x, m))       : your input data at timestep "t"
        a_prev (ndarray (n_a, m))   : Hidden state at timestep "t-1"
        c_prev (ndarray (n_a, m))   : Memory state at timestep "t-1"
        parameters (dict) :
            Wf (ndarray (n_a, n_a + n_x))    : Weight matrix of the forget gate
            bf (ndarray (n_a, 1))            : Bias of the forget gate
            Wi (ndarray (n_a, n_a + n_x))    : Weight matrix of the update gate
            bi (ndarray (n_a, 1))            : Bias of the update gate
            Wc (ndarray (n_a, n_a + n_x))    : Weight matrix of c~
            bc (ndarray (n_a, 1))            : Bias of c~
            Wo (ndarray (n_a, n_a + n_x))    : Weight matrix of the output gate
            bo (ndarray (n_a, 1))            : Bias of the output gate
            Wy (ndarray (n_y, n_a))          : Weight matrix of prediction y_hat
            by (ndarray (n_y, 1))            : Bias of prediction y_hat
                        
    Returns:
        a_next (ndarray (n_a, m))   : next hidden state timestep "t"
        c_next (ndarray (n_a, m))   : next memory state timestep "t" 
        yt_hat (ndarray (n_y, m))   : prediction at timestep "t"
        cache (tuple)               : contains (a_next, c_next, a_prev, c_prev, xt, parameters)
                                      needed for the backward pass

    Note:
        ft/it/ot stand for the forget/update/output gates
        cct stands for the candidate value (c~),
        c stands for the cell state (memory)
    """

    ## Retrieve parameters
    # Forget gate
    Wf = parameters["Wf"]
    bf = parameters["bf"]

    # Update gate
    Wi = parameters["Wi"]
    bi = parameters["bi"]

    # Candidate value c~
    Wc = parameters["Wc"]
    bc = parameters["bc"]

    # Output gate
    Wo = parameters["Wo"]
    bo = parameters["bo"]

    # Prediction y_hat
    Wy = parameters["Wy"]
    by = parameters["by"]

    ## Retrieve dimensions
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    ## Compute forward
    # Concatenate a_prev and xt, shape (n_a + n_x, m)
    concat = np.concatenate((a_prev, xt), axis=0)

    # Forget gate, shape (n_a, 1)
    ft = sigmoid(np.dot(Wf, concat) + bf)

    # Update gate, shape (n_a, 1)
    it = sigmoid(np.dot(Wi, concat) + bi)

    # Candidate value, shape (n_a, 1)
    cct = np.tanh(np.dot(Wc, concat) + bc)

    # Output gate
    ot = sigmoid(np.dot(Wo, concat) + bo)

    # Cell state
    c_next = ft*c_prev + it*cct

    # Hidden state
    a_next = ot * np.tanh(c_next)
    
    # prediction
    yt_hat = softmax(np.dot(Wy, a_next) + by)

    ## Cache + return
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)
    return a_next, c_next, yt_hat, cache

## 1.2 Forward Pass

<img src="./assets/LSTM_forwardpass.png" width="1100"/>


In [4]:
def lstm_forward(X, a0, parameters):
    """
    Arguments:
        X (ndarray (n_x, m, T_x)) : Input data for every time-step
        a0 (ndarray (n_a, m)) : Initial hidden state
        parameters (dict) :
            Wf (ndarray (n_a, n_a + n_x))    : Weight matrix of the forget gate
            bf (ndarray (n_a, 1))            : Bias of the forget gate
            Wi (ndarray (n_a, n_a + n_x))    : Weight matrix of the update gate
            bi (ndarray (n_a, 1))            : Bias of the update gate
            Wc (ndarray (n_a, n_a + n_x))    : Weight matrix of c~
            bc (ndarray (n_a, 1))            : Bias of c~
            Wo (ndarray (n_a, n_a + n_x))    : Weight matrix of the output gate
            bo (ndarray (n_a, 1))            : Bias of the output gate
            Wy (ndarray (n_y, n_a))          : Weight matrix of prediction y_hat
            by (ndarray (n_y, 1))            : Bias of prediction y_hat

    Returns:
        a (ndarray (n_a, m, T_x))        : Hidden states for every time-step
        Y_hat (ndarray (n_y, m, T_x))    : Predictions for every time-step
        caches ((list(cache_t), X))      : list of cache every timestep t, X for the backward pass
    """
    
    # Retrieve dim
    n_x, m, T_x = X.shape
    n_y, n_a = parameters["Wy"].shape
    
    # Initialize "a", "c" and "Y_hat"
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    Y_hat = np.zeros((n_y, m, T_x))
    
    # Initialize at and ct
    at = a0
    ct = np.zeros(at.shape)
    
    # loop over all time-steps
    caches = []
    for t in range(T_x):
        # Cell forward
        at, ct, yt, cache = lstm_cell_forward(X[:,:,t], at, ct, parameters)

        # Update params_t
        a[:,:,t] = at
        c[:,:,t]  = ct
        Y_hat[:,:,t] = yt

        # Append the cache into caches
        caches.append(cache)
        
    
    # store values needed for backward propagation in cache
    caches = (caches, X)
    return a, Y_hat, c, caches

# 2. Backpropagation

## 2.1 LSTM backward Cell

####  Gate derivatives

$$d \Gamma_o^{\langle t \rangle} = da_{next}*\tanh(c_{next}) * \Gamma_o^{\langle t \rangle}*(1-\Gamma_o^{\langle t \rangle})\tag{7}$$

$$d\tilde c^{\langle t \rangle} = dc_{next}*\Gamma_u^{\langle t \rangle}+ \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * i_t * da_{next} * \tilde c^{\langle t \rangle} * (1-\tanh(\tilde c)^2) \tag{8}$$

$$d\Gamma_u^{\langle t \rangle} = dc_{next}*\tilde c^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * \tilde c^{\langle t \rangle} * da_{next}*\Gamma_u^{\langle t \rangle}*(1-\Gamma_u^{\langle t \rangle})\tag{9}$$

$$d\Gamma_f^{\langle t \rangle} = dc_{next}*\tilde c_{prev} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * c_{prev} * da_{next}*\Gamma_f^{\langle t \rangle}*(1-\Gamma_f^{\langle t \rangle})\tag{10}$$

#### Parameter derivatives 

$$ dW_f = d\Gamma_f^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{11} $$
$$ dW_u = d\Gamma_u^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{12} $$
$$ dW_c = d\tilde c^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{13} $$
$$ dW_o = d\Gamma_o^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{14}$$

#### Parameter derivatives 

$$ da_{prev} = W_f^T*d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle}+ W_c^T * d\tilde c^{\langle t \rangle} + W_o^T * d\Gamma_o^{\langle t \rangle} \tag{15}$$

$$ dc_{prev} = dc_{next}\Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1- \tanh(c_{next})^2)*\Gamma_f^{\langle t \rangle}*da_{next} \tag{16}$$

$$ dx^{\langle t \rangle} = W_f^T*d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle}+ W_c^T * d\tilde c_t + W_o^T * d\Gamma_o^{\langle t \rangle}\tag{17} $$

In [5]:
def lstm_cell_backward(da_next, dc_next, cache):
    """
    Arguments:
        da_next (ndarray (n_a, m)) : Gradient of loss with respect to next hidden state
        dc_next (ndarray (n_a, m)) : Gradients of next cell state
        cache (tuple) : contain output of the LSTM cell forward pass

    Returns:
        gradients (dict) :
            dxt (ndarray (n_x, m))      : Gradients of input data
            da_prev (ndarray (n_a, m))  : Gradients of previous hidden state
            dc_prev (ndarray (n_a, m))  : Gradient of the previous memory state
            dWf (ndarray (n_a, n_a + n_x))      : Gradient of the forget gate weight matrix
            dWi (ndarray (n_a, n_a + n_x))      : Gradient of the update gate weight matrix
            dWc (ndarray (n_a, n_a + n_x))      : Gradient of the memory gate weight matrix
            dWo (ndarray (n_a, n_a + n_x))      : Gradient of the output gate weight matrix
            dbf (ndarray (n_a, 1))              : Gradient of the forget gate bias matrix
            dbi (ndarray (n_a, 1))              : Gradient of the update gate bias matrix
            dbc (ndarray (n_a, 1))              : Gradient of the memory gate bias matrix
            dbo (ndarray (n_a, 1))              : Gradient of the output gate bias matrix
    """

    # Retrieve values from cache
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache
    
    # Retrieve dim
    n_x, m = xt.shape 
    n_a, m = a_next.shape 
    
    # Compute Gate derivatives
    dit = (da_next * ot * (1 - np.tanh(c_next) ** 2) + dc_next) * cct * (1 - it) * it
    dft = (da_next * ot * (1 - np.tanh(c_next) ** 2) + dc_next) * c_prev * ft * (1 - ft)
    dot = da_next * np.tanh(c_next) * ot * (1 - ot)
    dcct = (da_next * ot * (1 - np.tanh(c_next) ** 2) + dc_next) * it * (1 - cct ** 2)

    # Compute Parameters derivatives
    dWf = np.dot(dft, np.concatenate((a_prev, xt), axis=0).T) # or use np.dot(dft, np.hstack([a_prev.T, xt.T]))
    dWi = np.dot(dit, np.concatenate((a_prev, xt), axis=0).T)
    dWc = np.dot(dcct, np.concatenate((a_prev, xt), axis=0).T)
    dWo = np.dot(dot, np.concatenate((a_prev, xt), axis=0).T)
    dbf = np.sum(dft, axis=1, keepdims=True)
    dbi = np.sum(dit, axis=1, keepdims=True) 
    dbc = np.sum(dcct, axis=1,keepdims=True) 
    dbo = np.sum(dot, axis=1, keepdims=True)  

    # Compute  previous hidden state derivatives
    da_prev = np.dot(parameters['Wf'][:,:n_a].T,dft) \
        + np.dot(parameters['Wi'][:,:n_a].T,dit) \
        + np.dot(parameters['Wc'][:,:n_a].T,dcct) \
        + np.dot(parameters['Wo'][:,:n_a].T,dot) 
    dc_prev = dc_next * ft \
        + ot * (1-np.square(np.tanh(c_next))) * ft * da_next 
    dxt = np.dot(parameters['Wf'][:,n_a:].T,dft) \
        + np.dot(parameters['Wi'][:,n_a:].T,dit) \
        + np.dot(parameters['Wc'][:,n_a:].T,dcct) \
        + np.dot(parameters['Wo'][:,n_a:].T,dot) 
    
    # Save gradients in dictionary
    gradients = {
        "dxt": dxt, "da_prev": da_prev, "dc_prev": dc_prev,
        "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi, "dWc": dWc, "dbc": dbc, "dWo": dWo,"dbo": dbo}
    return gradients

## 2.2 LSTM backward pass

In [6]:
def lstm_backward(da, caches):
    
    """
    Arguments:
    da (ndarray (n_a, m, T_x)) : Gradient of the hidden state
    cache (list of tuples) : contain output of the LSTM forward pass

    Returns:
    gradients (dict) :
            dxt (ndarray (n_x, m))      : Gradients of input data
            da_prev (ndarray (n_a, m))  : Gradients of previous hidden state
            dc_prev (ndarray (n_a, m))  : Gradient of the previous memory state
            dWf (ndarray (n_a, n_a + n_x))      : Gradient of the forget gate weight matrix
            dWi (ndarray (n_a, n_a + n_x))      : Gradient of the update gate weight matrix
            dWc (ndarray (n_a, n_a + n_x))      : Gradient of the memory gate weight matrix
            dWo (ndarray (n_a, n_a + n_x))      : Gradient of the output gate weight matrix
            dbf (ndarray (n_a, 1))              : Gradient of the forget gate bias matrix
            dbi (ndarray (n_a, 1))              : Gradient of the update gate bias matrix
            dbc (ndarray (n_a, 1))              : Gradient of the memory gate bias matrix
            dbo (ndarray (n_a, 1))              : Gradient of the output gate bias matrix
    """

    # Retrieve values from the first cache (t=1)
    (caches, x) = caches
    (a1, c1, a0, c0, f1, i1, cc1, o1, x1, parameters) = caches[0]
    
    # Retrieve dim
    n_a, m, T_x = da.shape
    n_x, m = x1.shape
    
    # zero gradients
    dx = np.zeros((n_x, m, T_x))
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))
    dc_prevt = np.zeros((n_a, m))
    dWf = np.zeros((n_a, n_a + n_x))
    dWi = np.zeros((n_a, n_a + n_x))
    dWc = np.zeros((n_a, n_a + n_x))
    dWo = np.zeros((n_a, n_a + n_x))
    dbf = np.zeros((n_a, 1))
    dbi = np.zeros((n_a, 1))
    dbc = np.zeros((n_a, 1))
    dbo = np.zeros((n_a, 1))
    
    # loop back over all timesteps
    for t in reversed(range(T_x)):
        # Compute gradient timestep t
        gradients = lstm_cell_backward(da[:,:,t] + da_prevt, dc_prevt, caches[t])

        # Update gradient at timestep t
        dx[:,:,t] = gradients["dxt"]
        dWf += gradients["dWf"]
        dWi += gradients["dWi"]
        dWc += gradients["dWc"]
        dWo += gradients["dWo"]
        dbf += gradients["dbf"]
        dbi += gradients["dbi"]
        dbc += gradients["dbc"]
        dbo += gradients["dbo"]

    # Set the first activation's gradient to the backpropagated gradient da_prev.
    da0 = gradients["da_prev"]

    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}
    
    return gradients